In [2]:
data = !wget https://www.dropbox.com/s/64ol9q9ssggz6f1/data_ford_price.xlsx

In [1]:
import pandas as pd

data = pd.read_excel('data\data_ford_price.xlsx')

<>:3: SyntaxWarning: invalid escape sequence '\d'
<>:3: SyntaxWarning: invalid escape sequence '\d'
C:\Users\Юрий\AppData\Local\Temp\ipykernel_60736\2623420869.py:3: SyntaxWarning: invalid escape sequence '\d'
  data = pd.read_excel('data\data_ford_price.xlsx')


In [2]:
data.head()

,price,year,condition,cylinders,odometer,title_status,transmission,drive,size,lat,long,weather
0,43900,2016,4,6,43500,clean,automatic,4wd,full-size,36.471500,-82.483400,59.0
1,15490,2009,2,8,98131,clean,automatic,4wd,full-size,40.468826,-74.281734,52.0
2,2495,2002,2,8,201803,clean,automatic,4wd,full-size,42.477134,-82.949564,45.0
3,1300,2000,1,8,170305,rebuilt,automatic,4wd,full-size,40.764373,-82.349503,49.0
4,13865,2010,3,8,166062,clean,automatic,4wd,NaN,49.210949,-123.114720,NaN


In [8]:
columns_to_change = ['cylinders', 'title_status', 'transmission', 'drive', 'size']

from sklearn.preprocessing import OneHotEncoder
 
one_hot_encoder = OneHotEncoder()
 
# 'учим' и сразу применяем преобразование к выборке, результат переводим в массив
data_onehot = one_hot_encoder.fit_transform(data[columns_to_change]).toarray()
 
# запишем полученные названия новых колонок в отдельную переменную
column_names = one_hot_encoder.get_feature_names_out(columns_to_change)
print(column_names)

['cylinders_3' 'cylinders_4' 'cylinders_5' 'cylinders_6' 'cylinders_8'
 'cylinders_10' 'title_status_clean' 'title_status_lien'
 'title_status_missing' 'title_status_rebuilt' 'title_status_salvage'
 'transmission_automatic' 'transmission_manual' 'transmission_other'
 'drive_4wd' 'drive_fwd' 'drive_rwd' 'drive_nan' 'size_compact'
 'size_full-size' 'size_mid-size' 'size_sub-compact' 'size_nan']


In [11]:
new_df = pd.DataFrame(columns=column_names, data=data_onehot)

In [13]:
new_df.shape

(7017, 23)

In [16]:
merged_data = pd.concat((data, new_df), axis=1)

In [18]:
merged_data = merged_data.drop(columns_to_change, axis=1)

In [19]:
merged_data.shape

(7017, 30)

In [25]:
data_copy = data.copy()

In [29]:
data_copy = data_copy[~data_copy['size'].isna()]

In [31]:
round((data_copy.shape[0] / data.shape[0]), 2)

0.78

In [33]:
merged_data.head()

,price,year,condition,odometer,lat,long,weather,cylinders_3,cylinders_4,cylinders_5,...,transmission_other,drive_4wd,drive_fwd,drive_rwd,drive_nan,size_compact,size_full-size,size_mid-size,size_sub-compact,size_nan
0,43900,2016,4,43500,36.471500,-82.483400,59.0,0.0,0.0,0.0,...,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0
1,15490,2009,2,98131,40.468826,-74.281734,52.0,0.0,0.0,0.0,...,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0
2,2495,2002,2,201803,42.477134,-82.949564,45.0,0.0,0.0,0.0,...,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0
3,1300,2000,1,170305,40.764373,-82.349503,49.0,0.0,0.0,0.0,...,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0
4,13865,2010,3,166062,49.210949,-123.114720,NaN,0.0,0.0,0.0,...,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0


In [34]:
from sklearn.linear_model import LinearRegression
from sklearn.metrics import r2_score
from sklearn.model_selection import train_test_split

In [87]:
data

,price,year,condition,cylinders,odometer,title_status,transmission,drive,size,lat,long,weather
0,43900,2016,4,6,43500,clean,automatic,4wd,full-size,36.471500,-82.483400,59.0
1,15490,2009,2,8,98131,clean,automatic,4wd,full-size,40.468826,-74.281734,52.0
2,2495,2002,2,8,201803,clean,automatic,4wd,full-size,42.477134,-82.949564,45.0
3,1300,2000,1,8,170305,rebuilt,automatic,4wd,full-size,40.764373,-82.349503,49.0
4,13865,2010,3,8,166062,clean,automatic,4wd,NaN,49.210949,-123.114720,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...
7012,22500,2015,3,6,23500,clean,automatic,rwd,full-size,32.680700,-117.169800,59.0
7013,5975,2005,2,8,0,clean,automatic,rwd,full-size,38.213303,-85.785762,50.0
7014,9999,2006,3,8,161514,clean,automatic,NaN,full-size,37.609783,-120.995406,59.0
7015,10900,2011,2,8,164000,clean,automatic,4wd,full-size,43.140600,-93.385000,47.0


In [106]:
X = data.drop('price', axis=1)
y = data['price']

In [107]:
X = X.dropna()

In [62]:
X.shape

(5202, 11)

In [108]:
y = y.iloc[X.index]

In [91]:
X.shape[0] == y.shape[0]

True

In [109]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=30)

In [110]:
from sklearn.preprocessing import OneHotEncoder

one_hot_encoder = OneHotEncoder()

X_train_onehot = one_hot_encoder.fit_transform(X_train[columns_to_change]).toarray()

X_test_onehot = one_hot_encoder.transform(X_test[columns_to_change]).toarray()

In [94]:
X_train.columns == X_test.columns

array([ True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True])

In [111]:
columns = one_hot_encoder.get_feature_names_out(columns_to_change)

In [112]:
X_train_one_hot_df = pd.DataFrame(X_train_onehot, columns=columns)

X_test_one_hot_df = pd.DataFrame(X_test_onehot, columns=columns)

In [135]:
X_train = X_train.reset_index().drop(['index'], axis = 1)
X_test = X_test.reset_index().drop(['index'], axis = 1)
 
y_train = y_train.reset_index().drop(['index'], axis = 1)
y_test = y_test.reset_index().drop(['index'], axis = 1)

X_train_new = pd.concat([X_train, X_train_one_hot_df], axis=1)
X_test_new = pd.concat([X_test, X_test_one_hot_df], axis=1)
 
X_train_new = X_train_new.drop(columns=columns_to_change)
X_test_new = X_test_new.drop(columns=columns_to_change)

In [136]:
X_test_one_hot_df.isna().sum()

cylinders_3               0
cylinders_4               0
cylinders_5               0
cylinders_6               0
cylinders_8               0
cylinders_10              0
title_status_clean        0
title_status_lien         0
title_status_missing      0
title_status_rebuilt      0
title_status_salvage      0
transmission_automatic    0
transmission_manual       0
transmission_other        0
drive_4wd                 0
drive_fwd                 0
drive_rwd                 0
size_compact              0
size_full-size            0
size_mid-size             0
size_sub-compact          0
dtype: int64

In [137]:
lr_model = LinearRegression()

lr_model.fit(X_train_new, y_train)

LinearRegression()

In [138]:
y_train_predict = lr_model.predict(X_train_new)

y_test_predict = lr_model.predict(X_test_new)


In [139]:
print(f'R2 Score for train = {r2_score(y_train, y_train_predict):.3f}')
print(f'R2 Score for test = {r2_score(y_test, y_test_predict):.3f}')

R2 Score for train = 0.647
R2 Score for test = 0.693
